In [2]:
import Pkg
Pkg.add("Oscar")
Pkg.add("Polymake")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [1]:
using Oscar
using Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [13]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [14]:
tv.SMOOTH

false

In [15]:
tv.SIMPLICIAL

true

In [16]:
tv.RAYS

pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2


In [17]:
tv.MAXIMAL_CONES

pm::IncidenceMatrix<pm::NonSymmetric>
{0 1}
{1 2}
{2 3}
{0 3}


In [18]:
function slicematrix(A::AbstractMatrix)
    return [A[i, :] for i in 1:size(A,1)]
end

slicematrix (generic function with 1 method)

In [19]:
function rowMinors(A::AbstractMatrix,S::Set)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(A,1)
        if i in S
            append!(outList,slices[i])
        end
    end
    return transpose(hcat(outList...))
end

rowMinors (generic function with 1 method)

In [20]:
findall(a -> a, tv.MAXIMAL_CONES[1,:])

2-element Array{Int64,1}:
 1
 2

In [21]:
# Get the rays of maximal cone i (tv.MAXIMAL_CONES[i,:] will get the incidence matrix booleans)
# findall gets the indices of the "true"s, then we take those specific rows
i = 3
coneRays = tv.RAYS[findall(bool -> bool, tv.MAXIMAL_CONES[i,:]),:]

pm::Matrix<pm::Rational>
0 -1
-1 1/2


In [57]:
# Seems to require cast to generic matrix in order to calculate rank
rank(Matrix(tv.RAYS))
# findfirst

2

In [59]:
rank(Matrix(coneRays))

2

In [66]:
function getConeRank(coneRayIndices, rayMatrix)
    coneRays = tv.RAYS[findall(bool -> bool, coneRayIndices),:]
    return rank(Matrix(coneRays))
end

getConeRank (generic function with 1 method)

In [72]:
badCone = Nothing

Nothing

In [80]:
function getDimension(X)
    return size(X.RAYS, 2)
end

getDimension (generic function with 1 method)

In [81]:
getDimension(tv)

2

In [ ]:
function makeSimplicial(X)
    Y = copy(X)
    while (true)
        # Maximal cones and ray matrix
        coneList = Y.MAXIMAL_CONES
        rayMatrix = Y.RAYS
        badCone = Nothing
        for i in 1:size(coneList,1)
            cone = coneList[i,:]
            if (getConeRank(cone, rayMatrix) != size(cone))
                badCone = cone
            end
        end
        
        if (badCone == Nothing)
            # All cones are linearly independent
            break
        else
            # Use some function to get dimension of the fan
            # Calculate codimension of cone
            c = 1 + getDimension(Y) - rank(Matrix(rayMatrix))
            i = 0
            # Find the first ray that is contained in more than one orbit
            # and subdivide at that ray, using toricBlowup
            
            # Macaulay2 code
            #edges := select (orbits (Y,c), r -> all(r, j -> member (j,s)));
            #find the orbits of Y of dimension c that are contained in s
            #while select (edges, r -> not member (s#i,r)) === 1 do i = i+1;
            #find the first ray that is contained in more than one orbit
            #Y = toricBlowup({s#i},Y) )
            #subdivide at that ray
        end
        # findfirst does not seem to work with the MAXIMAL_CONES array
        #k = findfirst(cone -> (getConeRank(cone, tv.RAYS) != size(cone)), eachrow(coneList))
    end
    return Y
end
        